In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn
import statsmodels.api as sm

### Data loading

In [9]:
df_raw = pd.read_excel('/home/yelhe/data/Acc/E_Trottis_Unfallbericht_2019-2022.xlsx',sheet_name=None)

In [10]:
df_raw.keys()

dict_keys(['Allgemeine Angaben', 'Infrastruktur', 'Bedingungen', 'Objekte und Personen', 'Beteiligungen 1', 'Beteiligungen 2', 'Unfälle nach Unfalltyp'])

In [12]:
dfaa = df_raw['Allgemeine Angaben']
dfin = df_raw['Infrastruktur']
dfbe = df_raw['Bedingungen']
dfop = df_raw['Objekte und Personen']
dfb1 = df_raw['Beteiligungen 1']
dfb2 = df_raw['Beteiligungen 2']

In [23]:
dfaa = dfaa.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Unfalltyp \nGruppe':'atgroup_number',
                              'Unnamed: 5': 'atgroup', # accident type group
                              'Unfalltyp':'at_number',
                              'Unnamed: 7':'at', # accident type
                              'G*':'G',
                              'SV*':'SV',
                              'LV*':'LV',
                              'Sach-\nschaden**':'pd', # property damage
                              'Hauptursache':'mc', # main cause
                              'Koord E':'koorde',
                              'Koord N':'koordn'})


In [24]:
dfaa = dfaa.drop(['Bemerkung'], axis = 1)

In [25]:
dfaa.columns

Index(['No', 'Nr', 'Datum', 'Zeit', 'atgroup_number', 'atgroup', 'at_number',
       'at', 'G', 'SV', 'LV', 'pd', 'mc', 'Achsname', 'koorde', 'koordn'],
      dtype='object')

In [14]:
dfin.columns

Index(['Nr.', 'Unfall-Nr.', 'Unfallstelle', 'Inner-/ausserorts',
       'Vortrittsregelung', 'Strassenart', 'Höchst-geschwindigkeit',
       'Strassen-beleuchtung', 'Bemerkung'],
      dtype='object')

In [28]:
dfin = dfin.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Unfallstelle':'asite', # accident site
                              'Inner-/ausserorts': 'inout', # in/out of town
                              'Vortrittsregelung':'rop', # right of way/regulation of priority
                              'Strassenart':'rt', # road type 
                              'Höchst-geschwindigkeit':'maxspeed', # maximum of speed
                              'Strassen-beleuchtung':'sl'}) # road lighting
dfin = dfin.drop(['Bemerkung'], axis = 1)

In [15]:
dfbe.columns

Index(['Nr.', 'Unfall-Nr.', 'Witterung', 'Lichtverhältnis', 'Strassenzustand',
       'Verkehrsaufkommen', 'Bemerkung'],
      dtype='object')

In [31]:
dfbe = dfbe.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Witterung':'weatherc', # weather condition
                              'Lichtverhältnis': 'lightc', # light condition
                              'Strassenzustand':'roadc', # road condition
                              'Verkehrsaufkommen':'trafficv'# traffic volume
                             })
dfbe = dfbe.drop(['Bemerkung'], axis = 1)

In [32]:
dfbe.columns

Index(['No', 'Nr', 'weatherc', 'lightc', 'roadc', 'trafficv'], dtype='object')

In [16]:
dfop.columns

Index(['Nr.', 'Unfall-Nr.', 'Objekt-Nr.', 'Fahrzeugart',
       'Blutalkohol-konzent. (‰)', 'Atemalkohol-konzent. (mg/l)',
       'Personenart', 'Unfallfolgen', 'Anprall', 'Alter', 'Bemerkung'],
      dtype='object')

In [34]:
dfop = dfop.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Objekt-Nr.':'object',  
                              'Fahrzeugart':'vt', # vehicle type
                              'Blutalkohol-konzent. (‰)': 'blal', # blood alcohol
                              'Atemalkohol-konzent. (mg/l)':'bral', # breath alcohol
                              'Personenart':'pt', # person type
                              'Unfallfolgen':'aconsequence', # accident consequence
                              'Anprall':'impact',
                              'Alter':'age'
                             })
dfop = dfop.drop(['Bemerkung'], axis = 1)

In [17]:
dfb1.columns

Index(['Nr.', 'Unfall-Nr.', 'Kinder-unfall*', 'Kinderunfall auf dem Schulweg*',
       'Senioren-unfall*', 'Anzahl Kinder', 'Anzahl Kinder auf dem Schulweg',
       'Anzahl Senioren', 'Anzahl Fussgänger oder FäG', 'Anzahl Fahrräder',
       'Anzahl Fahrräder mit elektr. Tretunterstützung bis 25km/h',
       'Anzahl Fahrräder mit elektr. Tretunterstützung bis 45km/h** ',
       'Bemerkung'],
      dtype='object')

In [41]:
dfb1 = dfb1.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Kinder-unfall*':'childa', # children accident  
                              'Kinderunfall auf dem Schulweg*':'childa_sch', # children accident way to school
                              'Senioren-unfall*': 'seniora', # senor accident
                              'Anzahl Kinder':'childn', # number of children 
                              'Anzahl Kinder auf dem Schulweg':'childn_sch', # number of children way to school 
                              'Anzahl Senioren':'seniorn', # number of senior 
                              'Anzahl Fussgänger oder FäG':'pen', # number of pedestrians 
                              'Anzahl Fahrräder':'bin', # number of bicycles 
                              'Anzahl Fahrräder mit elektr. Tretunterstützung bis 25km/h':'nb25', # number of e-bicycles with speed up to 25
                              'Anzahl Fahrräder mit elektr. Tretunterstützung bis 45km/h** ':'nb45' # number of e-bicycles with speed up to 45
                             })
dfb1 = dfb1.drop(['Bemerkung'], axis = 1)

In [18]:
dfb2.columns

Index(['Nr.', 'Unfall-Nr.', 'Motor-fahrräder',
       'Kleinmotorräder bis 50 cm3 und max. 4kW', 'Motorräder\n', 'Lastwagen',
       'Lieferwagen', 'Personen-wagen', 'Transport-mittel des ÖV',
       'Andere Fahrzeuge', 'Bemerkung'],
      dtype='object')

In [46]:
dfb2 = dfb2.rename(columns = {'Nr.':'No', 
                              'Unfall-Nr.':'Nr',
                              'Motor-fahrräder':'motorcycle', 
                              'Kleinmotorräder bis 50 cm3 und max. 4kW':'small_motorbike', 
                              'Motorräder\n':'motorbike', 
                              'Lastwagen':'truck', 
                              'Lieferwagen':'van', 
                              'Personen-wagen':'passengercar', # passenger cars
                              'Transport-mittel des ÖV':'ptvehicle', # public transport vehicle
                              'Andere Fahrzeuge':'ovehicle' # other vehicle
                             })
dfb2 = dfb2.drop(['Bemerkung'], axis = 1)

In [49]:
dfaa.to_csv('/home/yelhe/script/mt/dataraw/dfaa.csv', index = False)
dfin.to_csv('/home/yelhe/script/mt/dataraw/dfin.csv', index = False)
dfbe.to_csv('/home/yelhe/script/mt/dataraw/dfbe.csv', index = False)
dfop.to_csv('/home/yelhe/script/mt/dataraw/dfop.csv', index = False)
dfb1.to_csv('/home/yelhe/script/mt/dataraw/dfb1.csv', index = False)
dfb2.to_csv('/home/yelhe/script/mt/dataraw/dfb2.csv', index = False)


In [59]:
print(dfaa.shape, dfin.shape, dfbe.shape, dfop.shape, dfb1.shape, dfb2.shape)

(350, 16) (350, 8) (350, 6) (581, 10) (351, 12) (351, 10)


#### Preparation for regression model

##### Distribution check

In [68]:
print(tabulate(nt))

---  ----------------------  ----------------------  ----------------------  ----------------------
     mew                     meg                     ie                      cn
var  4.40727951207253e-09    9.085136206442949e-20   1.2845308127903771e-07  2.243297732859301e-42
min  2.0773249104189035e-06  1.7123063677699816e-06  0.0015720596740620998   1.336316185102982e-101
max  2.1456165459478802e-11  7.321700186197608e-29   1.049612113000862e-14   2.1158469778304915e-20
---  ----------------------  ----------------------  ----------------------  ----------------------


#### Regression analysis

##### Variables transformation